In [1]:
for ii in 1:6
    addprocs(["heathred"])
end

for ii in 1:4
    addprocs(["amon"], dir="")
    addprocs(["zeus"], dir="")
    addprocs(["jove"], dir="")
    addprocs(["ares"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(6)
workers()

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


28-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 18
 19
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29

In [2]:
using Iterators
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [3]:
@everywhere using RecursiveAutoencoders
@everywhere  using UnfoldingRAE
include("ClusterSoup.jl")

get_chunks (generic function with 1 method)

In [4]:
include("load_embeddings.jl")
LL,word_indexes, indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,268810)
Dict{String,Int64}
Array{String,1}


In [5]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;
@assert length(@pipe training_trees |> filter(x->typeof(x)===UTF8String,_))==0
@pz training_trees

training_trees		Array{Any,1}	(3913,)


In [ ]:
chunk_data(:training_trees, training_trees)

In [ ]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len],W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end


In [ ]:

function loss!(θ::Vector)  
    error("loss! not defined")
end

function loss_grad!(θ::Vector, storage::Vector) 
    error("loss_grad not defined")
end


#rae_outer = RAE(LL,word_indexes,indexed_words);
set_global(:rae_outer,rae_outer)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    for pid in workers()
        remotecall(pid, θv->unpack!(rae_outer, θv),θ) 
    end
    
    
    function loss_and_loss_grad_single(tree::(Any,Any))
        Δs, err = UnfoldingRAE.loss_and_loss_grad(rae_outer, tree)
        [pack(Δs...), err]
    end
    
    ret = prechunked_mapreduce(:training_trees, loss_and_loss_grad_single, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_trees)
    err/=length(training_trees)
    err
end

In [ ]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_MMA, :LD_CCSAQ, :LD_LBFGS, :LD_SLSQP, :LD_VAR2, :LD_VAR1, :LD_TNEWTON_RESTART
opt = Opt(:LD_MMA, length(pack(rae_outer)))

#ftol_abs!(opt,1e-9)
maxtime!(opt, 60)
min_objective!(opt, tracking_loss_and_loss_grad!)
(optf,optx,ret) = optimize!(opt,pack(rae_outer))


In [ ]:
pack(rae_outer) |> sizeof

In [ ]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,loss_and_loss_grad!)

res = optimize(f, pack(rae_outer), method=:l_bfgs, show_trace = true, store_trace = true, iterations = 2);
@printval res.f_calls 
@printval res.g_calls 
@printval res.x_converged 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.trace


In [ ]:
Profile.clear()

In [ ]:
θ=pack(rae_outer)
@time loss_and_loss_grad!(θ, zeros(size(θ)))